# Evaluate Equivalence between RBAs with different number of edges

## Evaluate the collected indicators

### Load the experiments

In [7]:
from classes.utils.customtext import CustomText
import pandas as pd

exp_path = "outputs/exp7_data_3_8_10/"
indicators_file = exp_path + 'data.csv'

df = pd.read_csv(indicators_file)
print("Archivo cargado exitosamente")

Archivo cargado exitosamente


In [9]:
num_registers = df.shape[0]
print(f"El DataFrame tiene {num_registers} registros.")
df.keys()

El DataFrame tiene 210 registros.


Index(['i_sample', 'n_local_networks', 'n_var_network', 'v_topology',
       'n_output_variables', 'n_clauses_function', 'n_edges',
       'n_local_attractors', 'n_pair_attractors', 'n_attractor_fields',
       'n_time_find_attractors', 'n_time_find_pairs', 'n_time_find_fields'],
      dtype='object')

### Filter by local network

In [2]:
# filter by the number of local network
n_local_networks = 8
df_filter = df[df['n_local_networks'] == n_local_networks]
num_registers = df_filter.shape[0]
print(f"Existe {num_registers} registros para las RBAs de {n_local_networks} redes locales.")

Existe 50 registros para las RBAs de 8 redes locales.


### Statistic comparative by edges

In [3]:
import pandas as pd

# Assuming your DataFrame is named 'df'
grouped_df = df_filter.groupby('n_edges')

# Calculate mean, median, standard deviation for each target column
summary_stats = grouped_df[['n_local_attractors', 'n_pair_attractors', 'n_attractor_fields']].agg(
    ['mean', 'median', 'std']
)

summary_stats

n_local_attractors                   n_pair_attractors         \
                      mean median        std              mean median   
n_edges                                                                 
8                     31.6   29.5  11.500725              75.6   41.5   
9                     35.1   32.0  13.312066             106.7   54.0   
10                    39.1   35.0  15.372775             145.9   69.0   
11                    41.9   38.0  16.960411             186.5   82.0   
12                    45.1   41.0  18.008948             231.2   99.5   

                    n_attractor_fields                       
                std               mean median           std  
n_edges                                                      
8         74.937307             8032.9    9.5  20381.124113  
9        111.464045             8075.3    5.5  20366.898526  
10       151.778677             8140.1    5.5  20346.735728  
11       209.513855             8405.7    2.5  20350.826268  
12       244.098250             8612.6    2.5  20675.599345

# Find the sample with more number of Stable Attractor Fields

In [4]:
max_sample = df_filter.loc[df_filter['n_attractor_fields'].idxmax(), 'i_sample']
max_sample

3

# Load the CBN objects for an specific sample

In [5]:
import pickle
import os
import matplotlib.pyplot as plt

# Define el segundo número en la ruta del archivo pickle
i_sample = max_sample
n_networks = 8   # Cambia esto según sea necesario

# Ruta base sin el número cambiante
base_path_cbn = f'outputs/exp7_data_3_8_10/pkl_cbn/cbn_{i_sample}_{n_networks}_'

# Número inicial y rango máximo
start_number = n_networks
max_number = 2 * start_number

# Lista de archivos pickle existentes
pickle_files = [f"{base_path_cbn}{i}.pkl" for i in range(start_number, max_number + 1) if os.path.exists(f"{base_path_cbn}{i}.pkl")]

l_o_cbn = []
# Iterador para cargar y mostrar las topologías
for path_cbn in pickle_files:
    with open(path_cbn, 'rb') as file:
        o_cbn = pickle.load(file)
        l_o_cbn.append(o_cbn)
print('CBN objects loads in l_o_cbn')

CBN objects loads in l_o_cbn


# Show the number of Stable attractor fields for Edge

In [6]:
from classes.utils.customtext import CustomText

CustomText.make_title(f"Sample: {i_sample}, Number of Local Networks: {n_networks}")
CustomText.print_stars()
for o_cbn in l_o_cbn:
    print(f"Number of Edges: {len(o_cbn.l_directed_edges)} - Attractor Fields: {o_cbn.get_n_attractor_fields()}")
    o_cbn.count_fields_by_global_scenes()

**************************************************
SAMPLE: 3, NUMBER OF LOCAL NETWORKS: 8
**************************************************
Number of Edges: 8 - Attractor Fields: 65536
COUNTING THE STABLE ATTRACTOR FIELDS BY GLOBAL SCENE
{'11111111': 65536}
Number of Edges: 9 - Attractor Fields: 65536
COUNTING THE STABLE ATTRACTOR FIELDS BY GLOBAL SCENE
{'111111111': 65536}
Number of Edges: 10 - Attractor Fields: 65536
COUNTING THE STABLE ATTRACTOR FIELDS BY GLOBAL SCENE
{'1111111111': 65536}
Number of Edges: 11 - Attractor Fields: 65536
COUNTING THE STABLE ATTRACTOR FIELDS BY GLOBAL SCENE
{'11111111111': 65536}
Number of Edges: 12 - Attractor Fields: 66561
COUNTING THE STABLE ATTRACTOR FIELDS BY GLOBAL SCENE
{'111111111111': 65536, '111010000110': 1024, '000000000000': 1}
